In [1]:
import nb_utils

In [2]:
import torch
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from mle import bootstrap_intrinsic_dim_scale_interval as mle

In [3]:
#Import R packages
import rpy2
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr
r_base = importr('base')
intdimr = importr('intrinsicDimension')

In [148]:
from lightning_data_modules import HaarDecomposedDataset, ImageDatasets, PairedDataset, SyntheticDataset, SyntheticPairedDataset, Synthetic1DConditionalDataset, SyntheticTimeSeries, SRDataset, SRFLOWDataset, CryptoDataset, KSphereDataset, MammothDataset, LineDataset

#from configs.mammoth.vesde import get_config
#from configs.ksphere.N_1.uniform import get_config
#from configs.line.vesde import get_config
from configs.ksphere.N_1.non_uniform_1 import get_config
#from configs.ksphere.N_1.non_uniform_075 import get_config
#from configs.ksphere.N_1.non_uniform_05 import get_config

config = get_config()

In [149]:
from lightning_data_modules.utils import create_lightning_datamodule
DataModule = create_lightning_datamodule(config)
DataModule.setup()
train_dataloader = DataModule.train_dataloader()

In [150]:
X=[]
for _, x in enumerate(train_dataloader):
    X.append(x)
X = torch.cat(X, dim=0).numpy()
X = pd.DataFrame(X)

In [151]:
print(X.shape)
X.head()

(40000, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.091895,0.063149,-0.057836,0.009851,-0.051552,-0.117169,0.000009,0.092844,-0.054788,0.086325,...,-0.013538,0.063213,0.052811,-0.100079,0.048334,0.063131,-0.146685,0.008719,0.077223,-0.141876
1,-0.155574,0.043521,0.054153,-0.016723,0.134282,-0.061614,0.125422,0.109618,0.048618,-0.074640,...,-0.069378,0.060734,0.049249,-0.201172,-0.025436,0.139699,-0.057406,-0.026968,-0.106947,-0.049060
2,-0.029920,-0.064131,-0.143349,0.001694,0.058575,0.015757,0.171826,0.057257,0.023520,0.237250,...,0.029356,-0.009491,-0.000153,-0.147717,0.027601,0.083605,0.077628,0.166580,0.047118,-0.117942
3,0.082422,-0.004682,-0.163978,0.054701,-0.202720,0.045587,-0.019879,-0.006567,-0.169330,0.218598,...,0.076120,0.027204,-0.020368,0.069578,0.091084,-0.112655,-0.002900,0.035211,0.132046,-0.053046
4,0.054473,-0.035465,-0.180297,0.030449,-0.116978,-0.011708,-0.030929,0.030583,-0.134036,0.280926,...,0.060791,-0.001906,-0.011042,0.032861,0.050465,-0.065118,-0.022054,0.113072,0.132170,-0.080701


# MLE

In [48]:
k = 5
mle_result = mle(X.to_numpy(),  k1=k, k2=k, average=True)[0]

TypeError: 'numpy.float64' object is not callable

In [47]:
mle_result

NameError: name 'mle_result' is not defined

# MLE R

In [152]:
data_r = X.to_numpy()
dim = X.shape[1]
k=5
#mle = intdimr.maxLikPointwiseDimEst(data_r, k=5).rx2('dim.est')
mle = intdimr.maxLikGlobalDimEst(data_r, k=k).rx2('dim.est')

In [153]:
mle

array([5.34525818])

# Local PCA

In [146]:
data_r = X
dim = X.shape[1]
lpca = intdimr.pcaLocalDimEst(data_r, 'FO').rx2('dim.est')

In [147]:
lpca

array([3], dtype=int32)

In [107]:
df = pd.DataFrame(columns=['line'], index = ['mle', 'lpca'])
df['line'].loc['mle'] = mle[0]
df['line'].loc['lpca'] = lpca[0]
df.index.name = 'method'

In [108]:
df

,line
method,
mle,0.0
lpca,32


In [109]:
df.to_csv('test.csv')

In [98]:
lpca[0]

IndexError: invalid index to scalar variable.